In [ ]:
# 03_baseline_model.ipynb - Cellule 1
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importer vos modules personnalisés
import sys
sys.path.append('..')  # pour accéder au dossier src/
from src.model import create_cnn_model
from src.visualization import plot_training_history, plot_confusion_matrix
from src.evaluate import evaluate_model  # nous allons créer ce fichier

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Cellule 2
train_dir = '../data/train'
val_dir = '../data/val'
test_dir = '../data/test'

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
NUM_CLASSES = 38  # à ajuster selon votre dataset (38 classes PlantVillage)
EPOCHS = 20       # vous pouvez augmenter si nécessaire

In [ ]:
# Cellule 3
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

In [ ]:
# Cellule 4
model = create_cnn_model(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES)
model.summary()

In [ ]:
# Cellule 5
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('../models/baseline_best.h5', save_best_only=True)
]

In [ ]:
# Cellule 6
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# Cellule 7
plot_training_history(history)

In [ ]:
# Cellule 8
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
# Cellule 9
# Récupérer les prédictions
test_generator.reset()
predictions = model.predict(test_generator)
pred_classes = np.argmax(predictions, axis=1)

# Vraies classes
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Matrice de confusion
cm = confusion_matrix(true_classes, pred_classes)
plot_confusion_matrix(cm, class_labels, title='Confusion Matrix - Baseline')

In [ ]:
# Cellule 10
print(classification_report(true_classes, pred_classes, target_names=class_labels))

In [ ]:
# Cellule 11
model.save('../models/baseline_model_final.h5')